# Accessing Open Cropland and landcover Maps

The following provides a simple example for how to access and view the cropland and landcover maps. 

To access these data you need to have an AWS account. 

In [ ]:
import boto3
import geopandas as gpd
import folium
from shapely.geometry import Polygon
import json
from IPython.display import display

## Datasets

Start by getting a list of datasets in the bucket. 

In [ ]:
session = boto3.Session()
s3_client = session.client('s3')
bucket_name = 'mappingafrica'
response = s3_client.list_objects_v2(Bucket=bucket_name)

objects = [obj["Key"] for obj in response["Contents"]]
objects

### Croplands

The croplands prefix contains field boundary maps for several African countries, stored as pmtiles (for a web-mapping application) and geoparquet (for analytics). We'll provide an example with geoparquet. 

Read the GeoParquet data into a GeoDataFrame

In [ ]:
s3_url = f"s3://mappingafrica/{objects[0]}" # congo geoparquet

gdf = gpd.read_parquet(s3_url)
gdf = gdf.dropna(subset=['geometry'])
gdf.head()

Display the cropland map of 2022 for Congo

In [ ]:
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True) 
bounds = gdf.total_bounds

center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=6, 
               tiles='cartodbpositron')

folium.GeoJson(gdf).add_to(m)

m.save('interactive_map.html')
m